In [1]:
import glob
import numpy as np
from ESN import ESN
import pickle
import pandas as pd
from sklearn.preprocessing import Imputer
import math

### Create ESN Network

In [2]:
def safe_arctanh(x):
    if x.ndim == 2:
        for row in np.arange(x.shape[0]):
            x[row, :] = [i - 1e-15 if i == 1 else i + 1e-15 if i == -1 else i for i in x[row, :]]
    elif x.ndim == 1:
        x = [i - 1e-15 if i == 1 else i + 1e-15 if i == -1 else i for i in x]
    return np.arctanh(x)

In [3]:
esn = ESN(n_inputs=28,
          n_outputs=3,
          n_reservoir=50,
          spectral_radius=0.95,
          sparsity=0,
          noise=0.001,
          teacher_forcing=True,
          activation_out=np.tanh,
          inverse_activation_out=safe_arctanh,
          random_state=10,
          print_state=True)

### Training

#### Load and pre-process data

In [12]:
training_files = glob.glob('train_data/*.csv')

first = True

for f in training_files:
    # read dataset
    train_ds = pd.read_csv(f, header=None)
    
    X = train_ds.iloc[:, :-4].values
    y = train_ds.iloc[:, -4:].values
    
    X_full = X if first else np.concatenate((X_full, X))
    y_full = y if first else np.concatenate((y_full, y))
    
    first = False
    
# Fill missing values with mean
imputer = Imputer(missing_values='NaN', strategy='mean', axis=0)
imputer = imputer.fit(X_full)
X_train = imputer.transform(X_full)

# Min and max values for specific parameters
maxSpeedX = X_full[:, 0].max()
minSpeedX = X_full[:, 0].min()
maxSpeedY = X_full[:, 1].max()
minSpeedY = X_full[:, 1].min()
maxRPM = X_full[:, 4].max()
maxWheelSpin = X_full[:, 5:9].max()

In [14]:
def normalize(x, mmin, mmax):
    return (x - mmin)/(mmax-mmin)

def normalize2(x, mmin, mmax, a=-1, b=1):
    return (b - a) * (x - mmin) / (mmax - mmin) + a

normalization = True
X_train = np.zeros(X_full.shape)
y_train = np.zeros(y_full.shape)

if normalization:    
    X_train[:,0] = normalize2(X_full[:,0], minSpeedX, maxSpeedX)  # speedX = range(search min, search max)
    X_train[:,1] = normalize2(X_full[:,1], minSpeedY, maxSpeedY)  # speedY = range(search min, search max)
    X_train[:,2] = normalize2(X_full[:,2], -180, 180)  # angle = range(-180, 180)
    X_train[:,3] = normalize2(X_full[:,3], -1, 6)  # currentGear = range(-1, 6)
    X_train[:,4] = normalize2(X_full[:,4], 0, maxRPM)  # RPM = range(0, search max)
    for i in np.arange(5, 9):
        X_train[:,i] = normalize2(X_full[:,i], 0, maxWheelSpin)  # *wheelSpin = range(0, search max)
    for i in np.arange(9, 28):
        X_train[:,i] = normalize2(X_full[:,i], 0, 200)  # *sensorValues = range(0, 200)
    y_train[:,0] = normalize2(y_full[:,0], -1, 6)  # gear = range(-1, 6)
    y_train[:,1] = normalize2(y_full[:,1], -1, 1)  # steering = range(-1, 1)
    # for acceleration and break, compute their difference and normalize it
    accel_brake = y_full[:,2] - y_full[:,3]
    y_train[:,2] = normalize2(accel_brake, -1, 1)  # accelerate-brake = range(-1, 1)
else:
    X_train = X_full
    y_train = y_full
    
y_train = np.delete(y_train, 3, axis=1)

#### Fit data to ESN

In [18]:
safe_arctanh(y_train)

/usr/local/miniconda3/lib/python3.6/site-packages/ipykernel_launcher.py:7: RuntimeWarning: invalid value encountered in arctanh
  import sys


array([[ -4.58145366e-01,   1.58530564e-02,   2.94298500e-03],
       [ -4.58145366e-01,   1.58530564e-02,   1.76163616e+01],
       [ -4.58145366e-01,   1.58530564e-02,   1.76163616e+01],
       ..., 
       [  4.58145366e-01,  -2.09567450e-03,   1.76163616e+01],
       [  4.58145366e-01,  -2.09505602e-03,   1.76163616e+01],
       [  4.58145366e-01,  -2.09401986e-03,   1.76163616e+01]])

In [16]:
pred_training = esn.fit(X_train, y_train)
pred_training

computing states...
fitting...


/usr/local/miniconda3/lib/python3.6/site-packages/ipykernel_launcher.py:7: RuntimeWarning: invalid value encountered in arctanh
  import sys


training error:
nan


array([[-0.99982149,         nan,         nan],
       [-0.98897728,         nan,         nan],
       [-0.95502203,         nan,         nan],
       ..., 
       [ 0.209983  ,         nan,         nan],
       [ 0.22198009,         nan,         nan],
       [ 0.21033457,         nan,         nan]])